In [1]:
import os
import pandas as pd
import cianparser
from datetime import datetime, timedelta

# Инициализация переменной для хранения имени старого CSV файла
old_csv_filename = None

# Функция для загрузки списка городов из файла cities.txt
def load_locations(filename="cities.txt"):
    locations = {}
    file_path = os.path.join(os.getcwd(), filename)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл {filename} не найден.")
    
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            city, code = line.strip().split(',')
            locations[city.strip()] = code.strip()
    
    return locations

# Функция для фильтрации данных по дате публикации
def filter_by_publication_date(df, years=8):
    current_date = datetime.now()
    min_date = current_date - timedelta(days=years * 365)

    if 'publication_date' in df.columns:
        df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')
        df = df[df['publication_date'] >= min_date]
    else:
        print("В данных отсутствует столбец 'publication_date'. Фильтрация невозможна.")
    
    return df

def get_real_estate_data(location, deal_type, property_type, min_area, max_area, pages, min_price=300000, max_price=100000000):
    global old_csv_filename
    
    try:
        parser = cianparser.CianParser(location=location)
    except Exception as e:
        print(f"Ошибка при инициализации парсера: {e}")
        return None

    data = None
    try:
        if property_type == 1:  # Квартира
            data = parser.get_flats(
                deal_type=deal_type,
                rooms=(1, 2, 3, 4),
                with_saving_csv=False,  # Изменено на False
                additional_settings={
                    "min_price": min_price,
                    "max_price": max_price,
                    "min_area": min_area,
                    "max_area": max_area,
                    "floor_range": (1, 40),
                    "start_page": 1,
                    "end_page": pages  # Использование параметра pages для end_page
                }
            )
        elif property_type == 2:  # Новостройка
            data = parser.get_newobjects(with_saving_csv=False)  # Изменено на False
        elif property_type == 3:  # Дома, коттеджи, участки
            data = parser.get_suburban(
                suburban_type="townhouse",
                deal_type=deal_type,
                with_saving_csv=False,  # Изменено на False
                additional_settings={"start_page": 1, "end_page": pages}  # Использование параметра pages для end_page
            )
    except Exception as e:
        print(f"Ошибка при сборе данных: {e}")
        return None

    if not data:
        print("Данные не найдены")
        return None

    data_df = pd.DataFrame(data)

    if old_csv_filename and os.path.exists(old_csv_filename):
        os.remove(old_csv_filename)
        print(f"Старый CSV файл удален: {old_csv_filename}")

    filtered_data_df = filter_by_publication_date(data_df, years=8)  # Использование параметра years для фильтрации

    # Определяем папку на основе типа сделки
    if deal_type == "sale":
        folder = "Sale"
    elif deal_type == "rent_long":
        folder = "Rent"
    else:
        print("Неверный тип сделки")
        return None

    # Проверка и создание папки
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Создана папка: {folder}")
    
    # Создаем имя файла с учетом типа сделки
    csv_filename = os.path.join(folder, f"{location}_{deal_type}_filtered_data.csv")
    
    # Сохраняем DataFrame в CSV файл
    filtered_data_df.to_csv(csv_filename, index=False)
    print(f"Новый DataFrame сохранен в файл {csv_filename}.")

    # Проверяем, сохранен ли файл в нужной папке
    if os.path.exists(csv_filename):
        print(f"Файл успешно сохранен в папке {folder}.")
    else:
        print(f"Ошибка! Файл не найден в папке {folder}.")

    old_csv_filename = csv_filename
    print(filtered_data_df.head())

    return filtered_data_df

def get_user_input(prompt, valid_options=None):
    while True:
        value = input(prompt).strip()
        if valid_options and value not in valid_options:
            print(f"Некорректный ввод. Допустимые значения: {', '.join(valid_options)}")
        else:
            return value

# Загрузка списка городов из файла cities.txt
try:
    locations = load_locations()
except FileNotFoundError as e:
    print(e)
    locations = {}

# Ввод данных пользователя
if locations:
    print("Доступные города:")
    for city in locations:
        print(f"{city} - код {locations[city]}")
    location = get_user_input("Введите название города: ", valid_options=locations.keys())

    deal_type_input = get_user_input("Введите тип сделки (1 - покупка, 2 - аренда): ", valid_options=["1", "2"])
    deal_type = "sale" if deal_type_input == "1" else "rent_long"

    property_type_input = get_user_input(
        "Введите тип объекта (1 - Квартира, 2 - Новостройка, 3 - Дом, участок): ",
        valid_options=["1", "2", "3"]
    )
    property_type = int(property_type_input)

    min_area = float(get_user_input("Введите минимальную площадь (кв.м): "))
    max_area = float(get_user_input("Введите максимальную площадь (кв.м): "))
    
    pages = int(get_user_input("Введите количество страниц фильтрации (например, 8): "))

    real_estate_data = get_real_estate_data(location, deal_type, property_type, min_area, max_area, pages)

    if real_estate_data is not None:
        print(f"Получено {len(real_estate_data)} записей по объектам недвижимости.")
else:
    print("Не удалось загрузить список городов.")


Доступные города:
Москва - код 1
Санкт-Петербург - код 2
Абакан - код '4638'
Анадырь - код '4648'
Архангельск - код '4658'
Астрахань - код '4660'
Барнаул - код '4668'
Белгород - код '4671'
Биробиджан - код '4682'
Благовещенск - код '4683'
Бронницы - код '4690'
Брянск - код '4691'
Великий Новгород - код '4694'
Владивосток - код '4701'
Владикавказ - код '4702'
Владимир - код '4703'
Волгоград - код '4704'
Вологда - код '4708'
Воронеж - код '4713'
Геленджик - код '4717'
Горно-Алтайск - код '4719'
Грозный - код '4723'
Дзержинский - код '4734'
Долгопрудный - код '4738'
Дубна - код '4741'
Екатеринбург - код '4743'
Жуковский - код '4750'
Звенигород - код '4756'
Иванов - код '4767'
Ижевск - код '4770'
Иркутск - код '4774'
Йошкар-Ола - код '4776'
Казань - код '4777'
Калининград - код '4778'
Калуга - код '4780'
Кемерово - код '4795'
Киров - код '4800'
Коломна - код '4809'
Королёв - код '4813'
Красноармейск - код '4817'
Краснодар - код '4820'
Краснознаменск - код '4822'
Красноярск - код '4827'
Кур

Введите название города:  Москва
Введите тип сделки (1 - покупка, 2 - аренда):  1
Введите тип объекта (1 - Квартира, 2 - Новостройка, 3 - Дом, участок):  1
Введите минимальную площадь (кв.м):  40
Введите максимальную площадь (кв.м):  80
Введите количество страниц фильтрации (например, 8):  20



                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=1&deal_type=sale&offer_type=flat&room1=1&room2=1&room3=1&room4=1&minprice=300000&maxprice=100000000

 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 5 %. Average price: 185 321 646 rub
 2 | 2 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 56. Progress ratio: 10 %. Average price: 218 622 768 rub
 3 | 3 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 84. Progress ratio: 15 %. Average price: 236 472 841 rub
 4 | 4 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 101. Progress ratio: 19 %. Average price: 249 012 179 rub
 5 | 5 page with list: [=